In [45]:
import numpy as np
np.set_printoptions(threshold=np.nan)
import info_theory as it
import err_exponents as err_exp
from collections import OrderedDict 
from scipy.stats import bernoulli

### Parameters for Causal Posterior Matching

In [69]:
k = 1
n = 2
p = 0.1 # cross_over probability
maxInputBits = 5

R = k/n
q = 1-p

capacity = it.capacity_bsc(p)
print('The capacity is {0}'.format(capacity))

The capacity is 0.5310044064107188


### Define functions for Causal Posterior Matching

In [50]:
# expand posterior vector
def expandPosterior(k, vec):
    new_vec = np.repeat(vec/(2**k), 2**k)
    return new_vec

# find the bin containing the median and decide which end point to choose
def findMedian(vec):
    cumsumVec = np.cumsum(vec)
    median_bin = [i for i in range(len(cumsumVec)) if cumsumVec[i] >=0.5][0]
    if median_bin == 0:
        leftToMedian = 0.5
        bin_ind = median_bin+1 # if left is automatically greater hence include the bin containing the median in 0
    else:
        leftToMedian = 0.5 - cumsumVec[median_bin-1]
        rightToMedian = vec[median_bin] - leftToMedian
        bin_ind = median_bin
        if leftToMedian >= rightToMedian: # if left is greater then include the bin containing the median in 0
            bin_ind = median_bin+1       
    return [median_bin, bin_ind] # bin_ind is the first bin which is mapped to input 1

# Message point bin from currently available bits
def findMessageBin(message_bits, count):
    binary_array = np.array([2**(count-1-i) for i in range(0,count+1)])
    message_bin = np.sum(binary_array*np.array(message_bits[0:count+1]))
    return message_bin

# obtain channel input by comparing message bin with median 
def channelInput(message_bin, bin_ind):
    x_input = 1
    if message_bin < bin_ind:
        x_input = 0   
    return x_input    
        
# obtain channel output        
def channelOutput(x_input, channel_flip_bits, count):
    y_output = (x_input+channel_flip_bits[count])%2
    return y_output

# update posterior vector
def updatePosterior(y_output, bin_ind, vec): # bin_ind is not included in 0 input
    vec[0:bin_ind] =  (q**(1-y_output))*(p**y_output)*vec[0:bin_ind]
    vec[bin_ind:] =  (p**(1-y_output))*(q**y_output)*vec[bin_ind:] 
    vec = vec/np.sum(vec)
    return vec


### Implementing Causal PM

In [70]:
# initialize posterior
posterior_vec = np.ones((1), dtype= float)

message_bits = bernoulli.rvs(0.5, size=maxInputBits)
channel_flip_bits = bernoulli.rvs(p, size=maxInputBits*n)

# Implementing Causal PM
for count in range(0, maxInputBits):
    # expand posterior for the new bit
    posterior_vec = expandPosterior(k,posterior_vec)
    print(posterior_vec)
    
    for i in range(0,n):
        # Message point bin from currently available bits
        message_bin= findMessageBin(message_bits, count)
    
        # find the bin containing the median and decide which end point to choose
        [median_bin, bin_ind] = findMedian(posterior_vec)
    
        # obtain channel input 
        x_input = channelInput(message_bin, bin_ind)
    
        # obtain channel output
        y_output = channelOutput(x_input, channel_flip_bits, count)
    
        # Update posterior
        posterior_vec =  updatePosterior(y_output, bin_ind, posterior_vec)
        print(posterior_vec)

[0.5 0.5]
[0.9 0.1]
[0.98780488 0.01219512]
[0.49390244 0.49390244 0.00609756 0.00609756]
[0.89778325 0.09975369 0.00123153 0.00123153]
[9.87507526e-01 1.21914509e-02 1.50511740e-04 1.50511740e-04]
[4.93753763e-01 4.93753763e-01 6.09572547e-03 6.09572547e-03
 7.52558700e-05 7.52558700e-05 7.52558700e-05 7.52558700e-05]
[8.97728655e-01 9.97476283e-02 1.23145220e-03 1.23145220e-03
 1.52031136e-05 1.52031136e-05 1.52031136e-05 1.52031136e-05]
[9.87500186e-01 1.21913603e-02 1.50510621e-04 1.50510621e-04
 1.85815582e-06 1.85815582e-06 1.85815582e-06 1.85815582e-06]
[4.93750093e-01 4.93750093e-01 6.09568016e-03 6.09568016e-03
 7.52553106e-05 7.52553106e-05 7.52553106e-05 7.52553106e-05
 9.29077909e-07 9.29077909e-07 9.29077909e-07 9.29077909e-07
 9.29077909e-07 9.29077909e-07 9.29077909e-07 9.29077909e-07]
[9.77723100e-02 8.79950790e-01 1.08635900e-02 1.08635900e-02
 1.34118395e-04 1.34118395e-04 1.34118395e-04 1.34118395e-04
 1.65578266e-06 1.65578266e-06 1.65578266e-06 1.65578266e-06
 1.65

In [68]:
sum(posterior_vec)

1.0